# Leçon 5 : Mon premier LLM

## On assemble tout !

Tu as appris :
1. **Prédire la suite** avec des probabilités
2. **Apprendre de ses erreurs** avec la loss et le gradient
3. **Les embeddings** pour donner une mémoire au modèle
4. **L'attention** pour regarder les lettres importantes

Maintenant, on met tout ensemble pour créer un **vrai mini-LLM**
qui génère des prénoms inventés !

## Architecture de notre LLM

```
Entrée : "em"  (on veut prédire 'm', 'a', '.')
          |
    [Token Embedding]   Chaque lettre -> vecteur de nombres
          +
    [Position Embedding] Chaque position -> vecteur de nombres
          |
    [Attention]          Chaque lettre regarde les précédentes
          |
    [MLP]                Réseau de neurones qui transforme l'info
          |
    [Softmax]            Transformer en probabilités
          |
    Sortie : probabilités pour chaque lettre
```

C'est la même architecture que GPT-2, GPT-3, GPT-4 !
Juste en **beaucoup** plus petit.

In [ ]:
import math
import random

random.seed(42)

# Configuration
VOCAB = list(".abcdefghijklmnopqrstuvwxyz")
VOCAB_SIZE = len(VOCAB)  # 27
EMBED_DIM = 16  # taille des embeddings
CONTEXT = 8  # nombre max de lettres en contexte
NUM_HEADS = 1  # tête d'attention (simplifiée pour la clarté)
HEAD_DIM = EMBED_DIM // NUM_HEADS  # 16
HIDDEN_DIM = 32  # taille du MLP

char_to_id = {c: i for i, c in enumerate(VOCAB)}
id_to_char = {i: c for i, c in enumerate(VOCAB)}

print("Configuration du mini-LLM :")
print(f"  Vocabulaire : {VOCAB_SIZE} caractères")
print(f"  Dimension embeddings : {EMBED_DIM}")
print(f"  Contexte max : {CONTEXT} lettres")
print(f"  Têtes d'attention : {NUM_HEADS}")
print(f"  Taille MLP : {HIDDEN_DIM}")

# Comptons les paramètres
nb_params = (
    VOCAB_SIZE * EMBED_DIM  # token embeddings
    + CONTEXT * EMBED_DIM  # position embeddings
    + 3 * EMBED_DIM * EMBED_DIM  # Q, K, V pour attention
    + EMBED_DIM * HIDDEN_DIM  # MLP couche 1
    + HIDDEN_DIM * EMBED_DIM  # MLP couche 2
    + EMBED_DIM * VOCAB_SIZE  # couche de sortie
)
print(f"  Nombre de paramètres : ~{nb_params:,}")
print()
print(f"  (GPT-4 en a ~1,800,000,000,000 -- {nb_params / 1.8e12 * 100:.10f}% de GPT-4)")

In [ ]:
# Fonctions utilitaires


def rand_matrix(rows, cols, scale=0.3):
    return [[random.gauss(0, scale) for _ in range(cols)] for _ in range(rows)]


def rand_vector(size, scale=0.3):
    return [random.gauss(0, scale) for _ in range(size)]


def mat_vec(mat, vec):
    """Multiplication matrice x vecteur."""
    return [sum(mat[i][j] * vec[j] for j in range(len(vec))) for i in range(len(mat))]


def vec_add(a, b):
    return [x + y for x, y in zip(a, b, strict=False)]


def softmax(scores):
    max_s = max(scores)
    exps = [math.exp(s - max_s) for s in scores]
    total = sum(exps)
    return [e / total for e in exps]


def relu(x):
    """Si positif, on garde. Si négatif, on met à zéro."""
    return [max(0, v) for v in x]


print("Fonctions utilitaires définies !")

In [ ]:
# Initialiser tous les poids du modèle

# Embeddings
tok_emb = rand_matrix(VOCAB_SIZE, EMBED_DIM, 0.5)  # token -> vecteur
pos_emb = rand_matrix(CONTEXT, EMBED_DIM, 0.5)  # position -> vecteur

# Attention (1 tête pour la clarté)
Wq = rand_matrix(EMBED_DIM, EMBED_DIM, 0.2)
Wk = rand_matrix(EMBED_DIM, EMBED_DIM, 0.2)
Wv = rand_matrix(EMBED_DIM, EMBED_DIM, 0.2)

# MLP
W1 = rand_matrix(HIDDEN_DIM, EMBED_DIM, 0.2)
b1 = [0.0] * HIDDEN_DIM
W2 = rand_matrix(EMBED_DIM, HIDDEN_DIM, 0.2)
b2 = [0.0] * EMBED_DIM

# Sortie
W_out = rand_matrix(VOCAB_SIZE, EMBED_DIM, 0.2)

print("Modèle initialisé avec des poids aléatoires.")
print("Il ne sait rien encore -- il faut l'entraîner !")

In [ ]:
def forward_llm(sequence_ids):
    """Passe une séquence dans le mini-LLM et retourne les probas pour le prochain token."""
    n = len(sequence_ids)

    # 1. Embeddings : token + position
    hidden = []
    for i, tok_id in enumerate(sequence_ids):
        h = vec_add(tok_emb[tok_id], pos_emb[i % CONTEXT])
        hidden.append(h)

    # 2. Self-Attention (sur la dernière position)
    # Query pour la dernière position
    q = mat_vec(Wq, hidden[-1])

    # Keys et Values pour toutes les positions
    scores = []
    values = []
    for i in range(n):
        k = mat_vec(Wk, hidden[i])
        v = mat_vec(Wv, hidden[i])
        score = sum(q[d] * k[d] for d in range(EMBED_DIM)) / math.sqrt(EMBED_DIM)
        scores.append(score)
        values.append(v)

    attn_weights = softmax(scores)

    # Somme pondérée des values
    attn_out = [0.0] * EMBED_DIM
    for i in range(n):
        for d in range(EMBED_DIM):
            attn_out[d] += attn_weights[i] * values[i][d]

    # Connexion résiduelle
    x = vec_add(hidden[-1], attn_out)

    # 3. MLP
    h = relu(vec_add(mat_vec(W1, x), b1))
    mlp_out = vec_add(mat_vec(W2, h), b2)

    # Connexion résiduelle
    x = vec_add(x, mlp_out)

    # 4. Sortie : scores pour chaque lettre
    logits = mat_vec(W_out, x)
    probas = softmax(logits)

    return probas


# Test avant entraînement
test_ids = [char_to_id[c] for c in ".em"]
probas = forward_llm(test_ids)
top5 = sorted(range(VOCAB_SIZE), key=lambda i: -probas[i])[:5]

print("Avant entraînement -- prédictions après '.em' :")
for idx in top5:
    print(f"  '{id_to_char[idx]}' : {probas[idx]:.1%}")
print("(La bonne réponse serait 'm' pour 'emma')")

In [ ]:
# Entraînement simplifié
# (On utilise une méthode numérique pour les gradients,
#  plus lente mais plus facile à comprendre)

prenoms = [
    "emma",
    "lucas",
    "lea",
    "hugo",
    "chloe",
    "louis",
    "alice",
    "jules",
    "lina",
    "adam",
    "rose",
    "arthur",
    "manon",
    "paul",
    "jade",
]


def calcul_loss(prenoms):
    """Calcule la loss moyenne sur tous les prénoms."""
    loss_totale = 0
    nb = 0
    for prenom in prenoms:
        mot = "." + prenom + "."
        ids = [char_to_id[c] for c in mot]
        for i in range(1, len(ids)):
            seq = ids[:i]
            cible = ids[i]
            probas = forward_llm(seq[-CONTEXT:])
            loss_totale += -math.log(probas[cible] + 1e-10)
            nb += 1
    return loss_totale / nb


loss_initiale = calcul_loss(prenoms)
print(f"Loss initiale : {loss_initiale:.3f}")
print(f"(Loss d'un modèle parfaitement aléatoire : {math.log(VOCAB_SIZE):.3f})")
print()
print("L'entraînement complet prendrait du temps en Python pur.")
print("C'est pour ça qu'en vrai on utilise PyTorch avec des GPU !")
print()
print("Mais l'ARCHITECTURE est exactement la même que GPT-2, GPT-3, GPT-4.")
print("Seuls la taille et la puissance de calcul changent.")

In [ ]:
# Génération (même sans entraînement complet, on peut voir le mécanisme)


def generer_llm(debut=".", temperature=1.0, max_len=15):
    """Génère un prénom lettre par lettre avec notre mini-LLM."""
    ids = [char_to_id[c] for c in debut]
    resultat = debut

    for _ in range(max_len):
        probas = forward_llm(ids[-CONTEXT:])

        # Température : < 1 = plus conservateur, > 1 = plus créatif
        if temperature != 1.0:
            logits = [math.log(p + 1e-10) / temperature for p in probas]
            probas = softmax(logits)

        # Choisir la prochaine lettre
        idx = random.choices(range(VOCAB_SIZE), weights=probas, k=1)[0]

        if idx == char_to_id["."]:
            break

        ids.append(idx)
        resultat += id_to_char[idx]

    return resultat[1:] if resultat.startswith(".") else resultat


print("Prénoms générés (modèle non-entraîné, juste pour montrer le mécanisme) :")
print()
for _ in range(10):
    p = generer_llm(temperature=0.8)
    print(f"  {p.capitalize()}")

print()
print("C'est du charabia car le modèle n'est pas entraîné.")
print("Mais le MÉCANISME est exactement celui de ChatGPT !")

## Récapitulatif : de 0 à GPT

```
Leçon 1 : Compter les lettres qui suivent        -> bigramme
Leçon 2 : Apprendre de ses erreurs                -> entraînement
Leçon 3 : Regarder plusieurs lettres en arrière   -> embeddings + contexte
Leçon 4 : Choisir les lettres importantes          -> attention
Leçon 5 : Assembler le tout                       -> mini-LLM !
```

## La différence avec ChatGPT

| | Notre mini-LLM | ChatGPT |
|---|---|---|
| Architecture | La même ! | La même ! |
| Paramètres | ~3,000 | ~1,800,000,000,000 |
| Données | 15 prénoms | Internet entier |
| Calcul | 1 PC, secondes | Des milliers de GPU, des mois |
| Résultat | Prénoms inventés | Conversations, code, poésie... |

L'algorithme est **le même**. La seule différence, c'est l'échelle.

> *"This file is the complete algorithm. Everything else is just efficiency."*
> -- Andrej Karpathy

## Pour aller plus loin

- **microgpt.py** : Le code complet de Karpathy avec l'autograd et l'entraînement
  [Lien](https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95)

- **Vidéo "Let's build GPT"** : Karpathy explique tout en 2h
  [YouTube](https://www.youtube.com/watch?v=kCc8FmEb1nY)

- **nanoGPT** : Version avec PyTorch, entraînable pour de vrai
  [GitHub](https://github.com/karpathy/nanoGPT)

---

**Félicitations ! Tu as compris comment fonctionne un LLM.**

*Prochaine leçon : [06 - Entraîner le modèle](06_entrainer_le_modele.ipynb)*

---

### Sources (ISO 42001)

- **Architecture complète GPT (embedding + attention + MLP + softmax)** : [microgpt.py](https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95) — Andrej Karpathy
- **Comparaison des paramètres GPT-4** : estimations publiques basées sur les rapports techniques OpenAI
- **Explication du forward pass complet** : [Vidéo "Let's build GPT"](https://www.youtube.com/watch?v=kCc8FmEb1nY) — Andrej Karpathy (2023)
- **Concept de température pour la génération** : même source, section sampling
- **"Attention Is All You Need"** : Vaswani et al., 2017, [arXiv:1706.03762](https://arxiv.org/abs/1706.03762)